In [1]:
import pandas as pd
import yfinance as yf
from pathlib import Path  
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
import talib
from os import path
import seaborn as sns
import numpy as np
import bs4 as bs
from urllib.request import Request, urlopen
from pulp import *

In [2]:
def reformat_ticker_dataframe(ticker_history):
    cols = {i:i.replace("_close","") for i in list(ticker_history.columns) if "_close" in i}
    ticker_history = ticker_history[[i for i in list(cols.keys())]]
    ticker_history = ticker_history.rename(columns=cols)
    return ticker_history        

def get_sp_index():
    sp_index_path =  Path('data/sp_index.xlsx')  
    if(sp_index_path.is_file()):
        df = pd.read_excel('data/sp_index.xlsx')
        df = df[["Company","Symbol","Weight","Price", "Chg"]]
    else:
        req = Request('https://www.slickcharts.com/sp500', headers={'User-Agent': 'Mozilla/5.0'})
        webpage = urlopen(req).read()
        soup = bs.BeautifulSoup(webpage,'lxml')
        table = soup.find('table', attrs={'class':'table table-hover table-borderless table-sm'})
        df = pd.read_html(str(table))[0]
        df = df[["Company","Symbol","Weight","Price","Chg"]]
        df.to_excel("data/sp_index.xlsx")
    return df


def get_ticker_historical(symbol_list):
    ticker_history_file = Path('data/ticker_history.xlsx')  
    if ticker_history_file.is_file():
        print("found")
        ticker_history = pd.read_excel('data/ticker_history.xlsx')
        ticker_history = ticker_history.set_index("Date")
        return reformat_ticker_dataframe(ticker_history.dropna())
    else:     
        print("not found")
        ticker_history = pd.DataFrame(list(), columns=[])   
        for i in symbol_list:
            ticker_df = yf.download(i, start="2015-03-30", end="2022-03-31")[["Close"]]    
            if len(ticker_df) > 500:        
                ticker_df = ticker_df.rename(columns={"Close": f"{i}_close"})
                ticker_history = ticker_df.join(ticker_history) 
                
        ticker_history.to_excel("data/ticker_history.xlsx")
        return reformat_ticker_dataframe(ticker_history.dropna())
    

In [8]:
sp_index = get_sp_index()
df = get_ticker_historical(list(set(sp_index["Symbol"])))
sp_index

found


,Company,Symbol,Weight,Price,Chg
0,Apple Inc.,AAPL,6.899424,165.72,0.43
1,Microsoft Corporation,MSFT,5.648914,281.89,2.06
2,Amazon.com Inc.,AMZN,3.575220,3067.01,32.89
3,Tesla Inc,TSLA,2.220094,1009.60,24.60
4,Alphabet Inc. Class A,GOOGL,2.052394,2568.00,33.40
...,...,...,...,...,...
500,IPG Photonics Corporation,IPGP,0.009450,97.67,0.38
501,Under Armour Inc. Class A,UAA,0.008207,16.04,-0.02
502,Under Armour Inc. Class C,UA,0.007884,14.84,-0.04
503,News Corporation Class B,NWS,0.007122,21.47,-0.01


In [43]:
df_2 = sp_index[["Symbol","Weight", "Price", "Chg"]]
df_2

,Symbol,Weight,Price,Chg
0,AAPL,6.899424,165.72,0.43
1,MSFT,5.648914,281.89,2.06
2,AMZN,3.575220,3067.01,32.89
3,TSLA,2.220094,1009.60,24.60
4,GOOGL,2.052394,2568.00,33.40
...,...,...,...,...
500,IPGP,0.009450,97.67,0.38
501,UAA,0.008207,16.04,-0.02
502,UA,0.007884,14.84,-0.04
503,NWS,0.007122,21.47,-0.01


In [86]:

sp_index
#Create the problem variable to contain problem data
stock_optimizer = LpProblem("StockMarketInvestment", LpMaximize)

#select list of tickers
tickers = df_2["Symbol"].tolist()

########################
## VARIABLES ##########
########################
ticker_vars = LpVariable.dicts("Stock_Problem", tickers, 0)

########################
##Objective function ##
########################
#minimize Sum(Food*Price of food)
stock_optimizer += lpSum(ticker_vars[i[1]] * i[0] for i in zip(df_2["Price"].tolist(),tickers)) 

#Set selected variables
stock_selected = LpVariable.dicts("selected", tickers,cat=LpBinary)

######################
## Constraints ######
######################
## loop through macros/energy requirements Calories through Iron

items = df_2["Chg"].tolist()
stock_optimizer += lpSum(ticker_vars[i[1]] * i[0]  for i in zip(items,tickers)) >= 1.0
     

items = df_2["Price"].tolist()
stock_optimizer += lpSum(ticker_vars[i[1]] * i[0]  for i in zip(items,tickers)) >= 1.0
stock_optimizer += lpSum(ticker_vars[i[1]] * i[0]  for i in zip(items,tickers)) <= 1000

for i in tickers:
    stock_optimizer +=  stock_selected["CTXS"] <= 1.0

######################
## Solve ############
######################
# The problem is solved using PuLP's choice of Solver
stock_optimizer.solve()

######################
## Result ############
######################
# The status of the solution is printed to the screen
print("----------------------------------------")
print("-----------------RESULTS----------------")
print("----------------------------------------")
print("Optimization status:", LpStatus[stock_optimizer.status])
print("----------------------------------------")

# Each of the variables is printed with it's resolved optimum value
for v in stock_optimizer.variables():
    if v.varValue > 0:
        print(v.name, "  {:.2f}".format(v.varValue))
        
print("----------------------------------------")
##The optimised objective function value is printed to the screen
print("Total Cost of Ingredients per can = $%.2f" % value(stock_optimizer.objective))
print("----------------------------------------")

----------------------------------------
-----------------RESULTS----------------
----------------------------------------
Optimization status: Optimal
----------------------------------------
Stock_Problem_CTXS   3.44
Stock_Problem_PSA   1.58
----------------------------------------
Total Cost of Ingredients per can = $1000.00
----------------------------------------


In [83]:
sp_index[sp_index["Symbol"] == "RTX"]

,Company,Symbol,Weight,Price,Chg
48,Raytheon Technologies Corporation,RTX,0.419524,104.3,0.03
